In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)

# from tensorflow.contrib.slim.nets import vgg

import sys
import glob

tf.reset_default_graph()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# make notebook display width scalable
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [11]:
slim_models_path = '/gpfs01/bethge/home/aecker/lab/projects/tf-models/slim'
if slim_models_path not in sys.path:
    sys.path.append(slim_models_path)
from datasets import imagenet
from preprocessing import vgg_preprocessing

In [12]:
import vgg, linvgg

In [13]:
IMAGENET_DATA = '/gpfs01/bethge/data/imagenet'

In [14]:
with tf.device('/cpu:0'):
    dataset = imagenet.get_split('validation', IMAGENET_DATA)
    provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, num_readers=5, shuffle=False)
    raw_image, raw_label = provider.get(['image', 'label'])
    image = vgg_preprocessing.preprocess_image(
        raw_image, 224, 224, is_training=False, resize_side_min=256, resize_side_max=256)
    label = tf.sub(raw_label, 1) # 1..1000 to 0..999

In [15]:
images, labels = tf.train.batch(
    [image, label], batch_size=32, capacity=256)

In [16]:
end_points = vgg.loss_vgg(images, subtract_mean=False, final_endpoint='fc8', gram_layers=['conv4/conv4_1'])
# end_points = vgg.loss_vgg(images, subtract_mean=False, final_endpoint='fc8')
# end_points = linvgg.loss_vgg(images, subtract_mean=False, final_endpoint='fc8', gram_layers=['conv4/conv4_1'])
predictions = tf.squeeze(end_points['fc8'], [1, 2])

In [17]:
metric_op, update_op = slim.metrics.streaming_recall_at_k(
    predictions, labels, k=5)

In [18]:
initial_op = tf.group(
    tf.initialize_all_variables(),
    tf.initialize_local_variables())

In [19]:
restorer = tf.train.Saver(slim.get_model_variables())
# restorer = tf.train.Saver(slim.get_model_variables(scope='vgg_19/fc'))
# restorer_normalized = tf.train.Saver(slim.get_model_variables(scope='vgg_19/conv'))

In [20]:
with tf.Session() as sess:
    sess.run(initial_op)
#     restorer.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_19.ckpt')
    restorer.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_normalized_complete.ckpt')
#     restorer.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_reduced_2.ckpt')
#     restorer_normalized.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_normalized.ckpt')
    with slim.queues.QueueRunners(sess):
        metric_values = slim.evaluation.evaluation(
            sess,
            num_evals=30,
            initial_op=None,
            eval_op=update_op,
            final_op=metric_op)

INFO:tensorflow:Executing eval ops
INFO:tensorflow:Executing eval_op 1/30
INFO:tensorflow:Executing eval_op 2/30
INFO:tensorflow:Executing eval_op 3/30
INFO:tensorflow:Executing eval_op 4/30
INFO:tensorflow:Executing eval_op 5/30
INFO:tensorflow:Executing eval_op 6/30
INFO:tensorflow:Executing eval_op 7/30
INFO:tensorflow:Executing eval_op 8/30
INFO:tensorflow:Executing eval_op 9/30
INFO:tensorflow:Executing eval_op 10/30
INFO:tensorflow:Executing eval_op 11/30
INFO:tensorflow:Executing eval_op 12/30
INFO:tensorflow:Executing eval_op 13/30
INFO:tensorflow:Executing eval_op 14/30
INFO:tensorflow:Executing eval_op 15/30
INFO:tensorflow:Executing eval_op 16/30
INFO:tensorflow:Executing eval_op 17/30
INFO:tensorflow:Executing eval_op 18/30
INFO:tensorflow:Executing eval_op 19/30
INFO:tensorflow:Executing eval_op 20/30
INFO:tensorflow:Executing eval_op 21/30
INFO:tensorflow:Executing eval_op 22/30
INFO:tensorflow:Executing eval_op 23/30
INFO:tensorflow:Executing eval_op 24/30
INFO:tensorflo

In [21]:
print(metric_values)

0.894792


In [21]:
# Create checkpoint for normalized net
saver = tf.train.Saver(slim.get_model_variables())
with tf.Session() as sess:
    sess.run(initial_op)
    restorer.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_19.ckpt')
    restorer_normalized.restore(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_normalized.ckpt')
    saver.save(sess, '/gpfs01/bethge/data/tf-model-checkpoints/vgg_normalized_complete.ckpt')


In [15]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph, max_const_size=32):
    graph_def = graph.as_graph_def()
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [16]:
import numpy as np
from IPython.display import clear_output, Image, display, HTML

g = tf.get_default_graph()
show_graph(g)